In [23]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [24]:
class crashstate(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [25]:
def step_1(state: crashstate)->crashstate:
    print("step 1 executed")
    return {"step1": "done", "input":state["input"]}

In [26]:
def step_2(state: crashstate)->crashstate:
    print("step 2 hangging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(30)
    return {"step2": "done"}

In [27]:
def step_3(state: crashstate)->crashstate:
    print("step 3 executed")
    return {"step3":"done"}

In [28]:
builder = StateGraph(crashstate)

builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()

workflow = builder.compile(checkpointer=checkpointer)

In [29]:
try:
    print("Running graph: please manually interrupt during step 2")
    workflow.invoke({"input":"start"}, config={"configurable": {"thread_id": 'tread-1'}})
except KeyboardInterrupt:
    print("kernel mannualy interrupted(crash simulated)")

Running graph: please manually interrupt during step 2
step 1 executed
step 2 hangging... now manually interrupt from the notebook toolbar (STOP button)
kernel mannualy interrupted(crash simulated)


In [30]:
workflow.get_state({"configurable":{"thread_id": 'thread-1'}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': 'thread-1'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [31]:
list(workflow.get_state_history({"configurable":{"thread_id": 'thread-1'}}))

[]

In [ ]:
final_state= workflow.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("final state", final_state)